In [14]:
import glob
from termcolor import colored

In [2]:
def export_test_file(files_dir, export=True):
    data = []
    for i, file in enumerate(sorted(glob.glob(files_dir+'*.tsv'))):
#         print(i, file)
        if 'test.txt' not in file:
            data.append(open(file).readlines())
    
    test_set, count = [], 0

    for essay_id,essay in enumerate(data):
#         print(essay_id)
        prev_sent_id = '0'

        for line in essay[1:]:
            sent_id, token_id, token, label = line.rstrip().split('\t')

            if sent_id != prev_sent_id:
                test_set.append('\n')
                count += 1

            if len(token) < 25 and 'www' not in token and '_NEW_LINE_' not in token:
                test_set.append('{} {}-claim\n'.format(token, label.split('-')[0]))

            prev_sent_id = sent_id

        test_set.append('\n')
        count += 1
        
    print(len(data), len(test_set), len(test_set)-count)

    if export:
        with open(files_dir+'test.txt', 'w') as writer:
            for line in test_set:
                writer.write(line)
    
    return data, test_set

## exporting merged files

In [4]:
_, _ = export_test_file('/Users/talhindi/Documents/data_wm/raw_data/',False)

0 0 0


([], [])

In [6]:
_, _ = export_test_file('/Users/talhindi/Documents/data_wm/arg_clean_45_1/',False)

45 26176 24910


In [17]:
export_test_file('/Users/talhindi/Documents/data_wm/arg_clean_45_2/')

45 38408 36546


In [3]:
export_test_file('/Users/talhindi/Documents/data_wm/wm_narrative/')

30 23184 21852


In [34]:
export_test_file('/Users/talhindi/Documents/claim_detection/data_newsplit/debanjan/train/')

291 111582 106484


In [35]:
export_test_file('/Users/talhindi/Documents/claim_detection/data_newsplit/debanjan/valid/')

31 12260 11719


In [36]:
export_test_file('/Users/talhindi/Documents/claim_detection/data_newsplit/debanjan/eval/')

45 26221 24955


## Picking examples for the presentations

In [3]:
def read_wm_essays(essays_dir):
     # read files
    data = []
    for file in sorted(glob.glob(essays_dir+'*.tsv')):
        data.append(open(file).readlines())


    essays_sent_token_label, tokens, labels = [], [], []

    for essay in data:
        prev_sent_id = '0'
        essay_sents, sent_token, sent_label = [], [], []
        doc_tokens, doc_labels = [], []

        for line in essay:
            sent_id, token_id, token, label = line.rstrip().split()
            
            if sent_id != prev_sent_id:
                essay_sents.append((sent_token, sent_label))
                sent_token, sent_label = [], []
                
            if len(token) < 25 and 'www' not in token:
                doc_tokens.append(token)
                doc_labels.append('{}-claim'.format(label.split('-')[0]))
                sent_token.append(token)
                sent_label.append('{}-claim'.format(label.split('-')[0]))
            
            prev_sent_id = sent_id
        
        essay_sents.append((sent_token, sent_label))
        essays_sent_token_label.append(essay_sents)
        tokens.append(doc_tokens)
        labels.append(doc_labels)

    essay_str, essay_str_sent = [], []
    for essay in essays_sent_token_label:
        
        sentences = []
        for essay_sent_tokens, essay_sent_labels in essay:
            sent = ' '.join(essay_sent_tokens)
    #         sent = sent.replace(" ' m", "'m")
    #         sent = sent.replace(" ' s", "'s")
    #         sent = sent.replace(" : ", ": ")
            sentences.append(sent)
        
        essay_str_sent.append(sentences)
        essay_str.append(' '.join(sentences))

    return {'essay_sent_token_label': essays_sent_token_label, 'tokens': tokens, 'labels': labels,
            'essay': essay_str, 'essay_sent': essay_str_sent}

In [4]:
wm1 = read_wm_essays('/Users/talhindi/Documents/data_wm/arg_clean_45_1/')
wm2 = read_wm_essays('/Users/talhindi/Documents/data_wm/arg_clean_45_2/')
wm_nr = read_wm_essays('/Users/talhindi/Documents/data_wm/wm_narrative/')

In [16]:
for i,essay in enumerate(wm1['essay']):
    if len(essay) < 2000:
        print(colored('{}-{}'.format(i,len(essay)), 'red'), end='\t')
    else:
        print('{}-{}'.format(i,len(essay)), end='\t')
    if (i+1) % 10 == 0:
        print()

print('\n')
for i,essay in enumerate(wm_nr['essay']):
    if len(essay) < 2000:
        print(colored('{}-{}'.format(i,len(essay)), 'red'), end='\t')
    else:
        print('{}-{}'.format(i,len(essay)), end='\t')
    if (i+1) % 10 == 0:
        print()

0-2786	1-2637	2-3353	3-5440	4-1813	5-2719	6-2301	7-2898	8-1770	9-2984	
10-4182	11-3838	12-2339	13-3240	14-4158	15-3108	16-3376	17-2561	18-1469	19-1088	
20-2943	21-2104	22-3744	23-2743	24-834	25-2133	26-4147	27-3777	28-3539	29-3434	
30-4017	31-2369	32-4118	33-3177	34-4005	35-1853	36-1776	37-2656	38-3219	39-2538	
40-2354	41-4821	42-2436	43-2491	44-2248	

0-6597	1-304	2-640	3-794	4-2515	5-1771	6-6962	7-3903	8-4167	9-809	
10-1061	11-147	12-449	13-2003	14-5468	15-117	16-2890	17-5024	18-4277	19-18384	
20-1796	21-4192	22-1018	23-2644	24-4747	25-2582	26-1601	27-6179	28-4151	29-5981	


In [65]:
len(wm1['essay_sent'][19])

14

In [24]:
for i,labels in enumerate(wm_nr['labels']):
    if 'B-claim' in labels:
        print(i, end=' ')

0 1 2 3 4 5 6 7 8 10 13 15 24 25 28 29 

In [31]:
wm_nr['essay'][3]

' token Overcoming Moving Schools and Homes According to the U.S. Census Bureau nearly 36 million people move every year in the U.S. - and , as a result , many children have to transfer schools , the Statesman Journal said in an article on tips for families with students transferring schools . That is about eleven percent of the United States population that moves per year . Moving schools can be fun and exciting or scary and sad , and at the same time being all of that . Especially when put on top of general house moving , too . The months before the realtor staked the For Sale sign in the crest of the ditch in our grassy green front yard , the spot where Kendall , Tyler , and Quinn had often marked as third base while playing baseball , they were tough months , on the whole family .'

In [31]:
for sent_tokens, sent_labels in wm2['essay_sent_token_label'][24]:
    for token in sent_tokens: print(token, end=' ')
    print()
    for label in sent_labels: print(label, end=' ')
    print('\n')

Should DNA modification be legal ? 
O-claim O-claim O-claim O-claim O-claim O-claim 

So DNA modification it sounds cool right . 
O-claim O-claim O-claim O-claim O-claim O-claim O-claim O-claim 

DNA modification is harmful . 
B-claim I-claim I-claim I-claim O-claim 

Most people believe that DNA modification started in China . 
O-claim O-claim O-claim O-claim B-claim I-claim I-claim I-claim I-claim O-claim 

But DNA modification did not start in China . 
B-claim I-claim I-claim I-claim I-claim I-claim I-claim I-claim O-claim 

DNA modification should be illegal , and here is why . 
B-claim I-claim I-claim I-claim I-claim O-claim O-claim O-claim O-claim O-claim O-claim 

DNA modification is dangerous . 
B-claim I-claim I-claim I-claim O-claim 

If a person messes up a DNA strand , it could kill a child . 
O-claim B-claim I-claim I-claim I-claim I-claim I-claim I-claim I-claim I-claim I-claim I-claim I-claim I-claim O-claim 

Would you want somebody to genetically modify your child ? 
O

In [44]:
import pandas as pd
import numpy as np

In [60]:
tokens = open('/Users/talhindi/Documents/claim_detection_wm/src/token_ids.txt').readlines()
LR_pred = pd.read_csv('/Users/talhindi/Documents/claim_detection_wm/skll/skll_wm_wm-arg/predictions/one_essay.tsv', sep='\t', header=None)
len(tokens), len(LR_pred)

(198, 198)

In [63]:
i, labels = 0, {1: 'B-claim', 2:'I-claim', 0:'O-claim'}
for row in LR_pred.iterrows():
    color_id = np.argmax(row[1][1:])
    if color_id == 1:
        print(colored(labels[np.argmax(row[1][1:])],'green'), tokens[i].split()[0])
    elif color_id == 2:
        print(colored(labels[np.argmax(row[1][1:])],'blue'), tokens[i].split()[0])
    else:
        print(labels[np.argmax(row[1][1:])], tokens[i].split()[0])
    i +=1

B-claim There
I-claim must
I-claim be
I-claim stricter
O-claim punishments
O-claim to
I-claim countries
O-claim or
O-claim people
O-claim who
O-claim break
I-claim them
O-claim .
B-claim War
O-claim is
O-claim always
O-claim brutal
O-claim ,
O-claim but
B-claim this
I-claim will
I-claim at
O-claim least
O-claim reduce
O-claim the
I-claim number
I-claim of
I-claim casualties
O-claim .
B-claim If
B-claim we
I-claim could
I-claim prevent
O-claim war
O-claim crimes
O-claim ,
O-claim then
B-claim many
I-claim more
I-claim people
I-claim would
I-claim live
O-claim through
I-claim a
O-claim conflict
O-claim .
B-claim Specific
O-claim weapons
O-claim being
O-claim banned
O-claim for
O-claim reasons
O-claim like
O-claim massive
O-claim property
O-claim damage
O-claim ,
O-claim unhuman
O-claim deaths
O-claim ,
O-claim can
I-claim seriously
O-claim injure
O-claim someone
O-claim ,
O-claim uncontainable
O-claim ,
O-claim and
O-claim uncontrollable
O-claim .
B-claim The
I-claim land
I-claim will
I-

## Archive

In [16]:
essays_sent_token_label, tokens, labels = [], [], []

for essay in data:
    prev_sent_id = '0'
    essay_sents, sent_token, sent_label = [], [], []
    doc_tokens, doc_labels = [], []

    for line in essay:
        sent_id, token_id, token, label = line.rstrip().split()
        
        if sent_id != prev_sent_id:
            essay_sents.append((sent_token, sent_label))
            sent_token, sent_label = [], []
            
        if len(token) < 25 and 'www' not in token:
            doc_tokens.append(token)
            doc_labels.append(label)
            sent_token.append(token)
            sent_label.append('{}-claim\n'.format(label.split('-')[0]))
        
        prev_sent_id = sent_id
    
    essay_sents.append((sent_token, sent_label))
    essays_sent_token_label.append(essay_sents)
    tokens.append(doc_tokens)
    labels.append(doc_labels)
    

In [14]:
len(essays_sent_token_label),sum([len(s[0]) for e in essays_sent_token_label for s in e])

(45, 27134)

In [19]:
sum([len(doc_tokens) for doc_tokens in tokens]), sum([len(doc_labels) for doc_labels in labels])

(27134, 27134)

In [7]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [8]:
essay_str, essay_str_sent = [], []
for essay in essays_sent_token_label:
    
    sentences = []
    for essay_sent_tokens, essay_sent_labels in essay:
        sent = ' '.join(essay_sent_tokens)
#         sent = sent.replace(" ' m", "'m")
#         sent = sent.replace(" ' s", "'s")
#         sent = sent.replace(" : ", ": ")
        sentences.append(sent)
    
    essay_str_sent.append(sentences)
    essay_str.append(' '.join(sentences))

In [9]:
essay_spacy = []
for essay in essay_str:
    essay_spacy.append(nlp(essay))

In [10]:
for essay_tok, essay_org, essay_sent in zip(essay_spacy,essays_sent_token_label, essay_str_sent):
    
    # asserting number of spacy tokens is equal to the original number of tokens
    assert len(essay_tok) == sum([len(sent_token) for sent_token, sent_label in essay_org])
    
#     count_sent = 0
#     for i, sent in enumerate(essay_tok.sents):
#         print(sent,'\n', essay_sent[max(0,i-1)], '\n')
#         count_sent += 1
#     print(count_sent, len(essay_org))
    
#     break

In [86]:
# tmp = nlp("You know what I ' m talking about the large fish that can kill you with one advance at your limbs .")
tmp = nlp("Take the most feared shark of them all the great white : From 1926 - 2011 great whites attacked Americans one hundred and six times when unprovoked and only thirteen of those attacks proved fatal .")
# tmp = nlp("Not only are the sharks eating their fellow brothers ' and sisters ' bodies but also the unfertilized eggs in a fit of embryonic cannibalism .")

print(len(tmp))
for sent in tmp.sents:
    print(sent, '\n*****')

36
Take the most feared shark of them all the great white : From 1926 - 2011 great whites attacked Americans one hundred and six times when unprovoked and only thirteen of those attacks proved fatal . 
*****


In [ ]:
tmp = nlp("You know what I'm talking about the large fish that can kill you with one advance at your limbs .")
# tmp = nlp("Take the most feared shark of them all the great white: From 1926 - 2011 great whites attacked Americans one hundred and six times when unprovoked and only thirteen of those attacks proved fatal .")
# tmp = nlp("Not only are the sharks eating their fellow brothers' and sisters' bodies but also the unfertilized eggs in a fit of embryonic cannibalism .")

print(len(tmp))
for sent in tmp.sents:
    print(sent, '\n*****')
for token in tmp:
    print('*{}*'.format(token), end=' ')

In [60]:
"You know what I ' m talking about the large fish that can kill you with one advance at your limbs .".find("'")

16

In [61]:
"You know what I ' m talking about the large fish that can kill you with one advance at your limbs .".replace(" '", "'")

"You know what I' m talking about the large fish that can kill you with one advance at your limbs ."